In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from typing import Iterable, Set, Tuple
import pandas as pd
import glob
import os
import re
import logging

log = logging.getLogger("debug")
log.setLevel(logging.DEBUG)

In [2]:
## constants
results_folder = "/home/pablo/results/seldonstar_1" 
cleanup_base_dir = "/tmp"

## Cargando los resultados de evaluar las quieres de evaluacion

In [3]:
# Take as input results folder, and assume that there exists a `v0` and `worse` folder with the evaluation results
v0_dir = "v0/"
boost_dir = "worse/"

results_boost = {}
results_v0 = {}

for f in glob.glob(os.path.join(results_folder, boost_dir, "*.csv")):
    results_boost[os.path.basename(f)] = pd.read_csv(f)

for f in glob.glob(os.path.join(results_folder, v0_dir, "*.csv")):
    results_v0[os.path.basename(f)] = pd.read_csv(f)

print("Read total %s dataframes" % (len(results_boost) + len(results_v0)))

Read total 100 dataframes


In [4]:
# Post-processing required for results:
# - remove the leading /tesis/tmp/*/ from the filePathSource and filePathSink columns
# - drop score
# - drop the origin worse results

# Post-processing results
# Clean filePath* columns
def cleanup_filePath_col(value: str) -> str:
    # /tesis/tmp/78fi2yky
    leading_replace_pat = "%s/[a-zA-Z0-9]+/" % (cleanup_base_dir)
    return re.sub(leading_replace_pat, "", value)

# This bit below will cleanup the /tesis/tmp/[a-zA-Z0-9]+ bit of the path column in the results
for df in results_boost.values():
    # Cleanup filePath* columns
    df['filePathSource'] = df['filePathSource'].map(cleanup_filePath_col)
    df['filePathSink'] = df['filePathSink'].map(cleanup_filePath_col)

for df in results_v0.values():
    df['filePathSource'] = df['filePathSource'].map(cleanup_filePath_col)
    df['filePathSink'] = df['filePathSink'].map(cleanup_filePath_col)

# utility function used for hashing each row in a consistent way, so they
# can be operated in sets
def hash_tuple(x: Tuple):
    return "#".join([
        str(v) for v in x
    ])

# Now, generate the a set for worse, v0 and boosted alone
v0 = set()
worse = set()
boosted = set()

# cleanup of each dataframe, dropping columns that will not give repeatable results between
# evaluation sets.
def cleanup(df):
    return df.drop(["score", "origin"], axis=1)

def calculate_df_intersect(a,b):
    sa = set(cleanup(a).apply(lambda x: hash_tuple(tuple(x)), axis=1))
    sb = set(cleanup(b).apply(lambda x: hash_tuple(tuple(x)), axis=1))
    return sa.intersection(sb)

# Calculate v0 set
for df in results_v0.values():
    # Drop non hashed columns
    df = cleanup(df)
    # hash and add to set
    v0 = v0 | set(df.apply(lambda x: hash_tuple(tuple(x)), axis=1))

# Calculate boosted
for df in results_boost.values():
    # filter out boosted
    df = df[df['origin'] == 'boosted']
    df = cleanup(df)
    boosted = boosted | set(
        df.apply(lambda x: hash_tuple(tuple(x)), axis=1))

# Calculate worse
for df in results_boost.values():
    # filter out boosted
    df = df[df['origin'] == 'worse']
    df = cleanup(df)
    worse = worse | set(df.apply(lambda x: hash_tuple(tuple(x)), axis=1))

all_results = v0 | worse | boosted

print("Result sets sizes: Worse %d, Boosted %d, V0 %d, All %d" %
         (len(worse), len(boosted), len(v0), len(all_results)))

Result sets sizes: Worse 542, Boosted 79, V0 480, All 978


## Analizo para db los resultados de las evaluaciones en worse, boosted y v0

In [5]:
counts = []
for n in results_boost.keys():
    intersect_size = len(calculate_df_intersect(results_boost[n], results_v0[n]))
    r_boost = results_boost[n]
    total_worse = r_boost[r_boost['origin'] == 'worse'].shape[0]
    total_boosted = r_boost[r_boost['origin'] == 'boosted'].shape[0]
    counts.append([n, total_worse, total_boosted, results_v0[n].shape[0], intersect_size])
df = pd.DataFrame(counts, columns=["name", "worse rows", "boosted rows", "v0 rows", "intersect"])
df

,name,worse rows,boosted rows,v0 rows,intersect
0,codejockie_maintenance-tracker_e6936f655a5c15a...,12,5,7,7
1,atulmy_wispy_706654487f865ba904c8292b552bfa273...,0,0,0,0
2,alexloof_next-graphql-blog_d7e5fb48488e78ee0dc...,0,0,0,0
3,bailicangdu_node-fs_1067a64972a11dc4372450b66e...,0,0,0,0
4,cyantarek_django-microservices_205acee4f006b2d...,1,1,0,0
5,alexyoung_nodepad_68357e774e2c971b39f9dec343a5...,5,0,3,0
6,alexzeitler_mongoose-schema-reference-sample_c...,0,0,0,0
7,danialk_passportjs-authentication_af6f6912a3cd...,1,0,1,1
8,brickyang_egg-mongo-native_e987e228efd447bb9b5...,0,0,0,0
9,cimrie_mongomem_b08739f9e9d1bac8a161980074d6b6...,0,0,0,0


In [8]:
df[df["worse rows"]==0].shape

(25, 5)

In [6]:
# set(results_v0['FurutaTakuto_infoVis2017_7b7aa8999b4233eb2b9e5fc7d3ba6488b80ed141.csv'].apply(lambda t: hash_tuple(t), axis=1))

# Helper function to make a set iterable in a repeatable order. Used for generating
# the item sets required by the score calculation functions in sklearn

def repeatable_for_each_set(s: Set[int]) -> Iterable[int]:
    return sorted(list(s))

all_ordered = repeatable_for_each_set(all_results)

y_worse = [
    int(it in worse)
    for it in all_ordered
]

y_pred = [
    int(it in (boosted.union(worse)))
    for it in all_ordered
]

y_true = [
    int(it in v0)
    for it in all_ordered
]

precision = precision_score(y_true, y_worse, pos_label=1)
recall = recall_score(y_true, y_worse, pos_label=1)
accuracy = accuracy_score(y_true, y_worse)
print("WORSE")
print("Score results: Precision: %.4f. Recall: %.4f. Accuracy: %.4f" % (precision, recall, accuracy))

precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
accuracy = accuracy_score(y_true, y_pred)
print("BOOSTED")
print("Score results: Precision: %.4f. Recall: %.4f. Accuracy: %.4f" % (precision, recall, accuracy))

WORSE
Score results: Precision: 0.0812. Recall: 0.0917. Accuracy: 0.0450
BOOSTED
Score results: Precision: 0.0812. Recall: 0.0917. Accuracy: 0.0450
